In [1]:
import math
import pandas as pd
import numpy as np
df = pd.read_csv('condo_res_join_v1.csv');
df_last = df[:100000]

df_buildings = df_last[
    ['PID',
     'ST_NUM',
    'ST_NAME',
    'ST_NAME_SUF',
    'UNIT_NUM',
    'ZIPCODE']
]


/Users/gabor/Projects/OurHome/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
def reformat_building_number(bldg):
    if bldg.count(' -') == 1:
        return bldg.replace(' -','-')
    if bldg == '12-Nov':
        return '11-12'
    if bldg == '10-Sep':
        return '9-10'
    if bldg == '10-Feb':
        return '2-10'
    if bldg == '  18-20':
        return '18-20'
    if bldg == '37 A 37C':
        return '37A-37C'
    if bldg == '4-Feb':
        return '2-4'
    if bldg == '12-Aug':
        return '8-12'
    if bldg == '14-Dec':
        return '12-14'
    if bldg == '7-Jan':
        return '1-7'
    if bldg == '10-Feb':
        return '2-10'
    if bldg == '7-Jan':
        return '1-7'
    if bldg == '7-May':
        return '5-7'
    if bldg == '10-Aug':
        return '8-10'
    if bldg == '3-Jan':
        return '1-3'
    if bldg == '12-Oct':
        return '10-12'
    if bldg == '5-Mar':
        return '3-5'
    if bldg == '9-Jul':
        return '7-9'
    if bldg == '4-Mar':
        return '3-4'
    if bldg == '11-Sep':
        return '9-11'
    if bldg == '2-Jan':
        return '1-2'
    if bldg.count(' ') == 1:
        return bldg.replace(' ','-')
    return bldg

df_buildings['ST_NUM'] = df_buildings.apply(lambda row: reformat_building_number(row['ST_NUM']), axis=1)

fixable=0
iempty=0
counter=0
nnum=0
for bldg_num in df_buildings['ST_NUM']:
    if not bldg_num or bldg_num == '' or bldg_num.count(' ') == len(bldg_num):
        iempty += 1
        continue
    if bldg_num.count(' ') == 1 or bldg_num.count(' - ') == 1:
        fixable += 1
    else:
        if bldg_num in ['  60R','4-Aug','6-Feb','6-Apr','8-Jun','13-Nov','18-Oct',
                        '11-Jan', '8-Jan','14-Aug',]:
            fixable += 1
            continue
            
        if bldg_num.isnumeric():
            counter += 1
        else:
            nnum += 1
            # print("{} {} ".format(bldg_num, bldg_num.count(' ')))

print("Empty: {} NonNumeric:{}".format(iempty, nnum))


Empty: 175 NonNumeric:11267


/Users/gabor/Projects/OurHome/venv/lib/python3.6/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
# Fix the name suffix, from NaN (which is float) to string
# Roads like "Riverway" and "Broadway" don't have suffix
# designation, so it is just better to leave it off, while
# cleaning large amounts of data.
df_buildings['ST_NAME_SUF'] = df_buildings.apply(lambda r: '' if isinstance(r['ST_NAME_SUF'], float) else r['ST_NAME_SUF'] , axis=1)

/Users/gabor/Projects/OurHome/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# Zip should be a 0 filled, left pad string of the original 
# number, and not a float.
df_buildings['Zip'] = df_buildings.apply(lambda r: format(int(r['ZIPCODE']), '05'), axis=1)

In [ ]:
import re
# Combine the columns representing the Address part of the input data for geocoding.
df_buildings['ADDRESS'] = df_buildings.apply(lambda r: r['ST_NUM']+" " +r['ST_NAME']+" " +r['ST_NAME_SUF'], axis=1)
# Strip duplicate whitespace
df_buildings['ADDRESS'] = df_buildings.apply(lambda r: re.sub(r'\s+', ' ',r['ADDRESS']).strip(), axis=1)

In [ ]:
# Create new dataframe from the address, and add some static data
# to it, since it is needed for the geo-coding service.
df_uaddress = df_buildings.drop_duplicates(subset='ADDRESS', keep="first")
df_uaddress['City'] = df_uaddress.apply(lambda r: 'Boston', axis=1)
df_uaddress['State'] = df_uaddress.apply(lambda r: 'MA', axis=1)

In [ ]:
# Output the dataframe as a CSV file, ready to be injested
# by the Census geocoding service.
print("The size of the unique address file is {}".format(df_uaddress['ADDRESS'].count()))
number_of_chunks = math.ceil(df_uaddress['ADDRESS'].count()/9500)
for id, df_i in  enumerate(np.array_split(df_uaddress, number_of_chunks)):
    output_filename = '../data/buildings_{}.csv'.format(id)
    print("Writing output file {}".format(output_filename))
    df_i.to_csv(output_filename, columns=['ADDRESS','City','State','Zip'], header = False)